In [177]:
import pandas as pd
import numpy as np

In [178]:
training=pd.read_csv('house-prices-advanced-regression-techniques/train.csv',index_col='Id')
test_x=pd.read_csv('house-prices-advanced-regression-techniques/test.csv',index_col='Id')
train_x=training.iloc[:,:-1]
train_y=training.iloc[:,-1]
test_y=pd.read_csv('house-prices-advanced-regression-techniques/sample_submission.csv',index_col='Id')
data=pd.concat([train_x,test_x],keys=["train","test"])
data[['MSSubClass','MoSold','OverallCond','OverallQual']]=data[['MSSubClass','MoSold','OverallCond','OverallQual']].astype('object')
data=data.fillna({'Alley': 'NA', 'FireplaceQu': 'NA', 'PoolQC': 'NA', 'Fence': 'NA', 'MiscFeature': 'NA',
                                         'BsmtQual': 'NA', 'BsmtCond': 'NA', 'BsmtExposure': 'NA', 'BsmtFinType1': 'NA', 'BsmtFinType2': 'NA',
                                         'GarageType': 'NA', 'GarageFinish': 'NA', 'GarageQual': 'NA', 'GarageCond': 'NA'})
na_l=data.columns[data.isna().any()].tolist()
me=training[na_l].select_dtypes("float64").median()
for item in training[na_l].select_dtypes("object"):
    tem=data[item].value_counts()
    tem1=tem.idxmax()
    data[item]=tem1
for item in me.index:
    data[item]=data[item].replace(np.nan,me[item])
train_x=data.loc["train"]
test_x=data.loc["test"]
training=train_x.join(train_y)

In [179]:
c=training.corr()
c['SalePrice'].sort_values()

KitchenAbvGr    -0.135907
EnclosedPorch   -0.128578
YrSold          -0.028923
LowQualFinSF    -0.025606
MiscVal         -0.021190
BsmtHalfBath    -0.016844
BsmtFinSF2      -0.011378
3SsnPorch        0.044584
PoolArea         0.092404
ScreenPorch      0.111447
BedroomAbvGr     0.168213
BsmtUnfSF        0.214479
BsmtFullBath     0.227122
LotArea          0.263843
HalfBath         0.284108
OpenPorchSF      0.315856
2ndFlrSF         0.319334
WoodDeckSF       0.324413
LotFrontage      0.334771
BsmtFinSF1       0.386420
GarageYrBlt      0.466754
Fireplaces       0.466929
MasVnrArea       0.472614
YearRemodAdd     0.507101
YearBuilt        0.522897
TotRmsAbvGrd     0.533723
FullBath         0.560664
1stFlrSF         0.605852
TotalBsmtSF      0.613581
GarageArea       0.623431
GarageCars       0.640409
GrLivArea        0.708624
SalePrice        1.000000
Name: SalePrice, dtype: float64